In [6]:
import os
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache
import numpy as np
import h5py
import time
import nrrd

drive_path = os.path.join(os.getenv('HOME'), 'work/allen/data/sdk_new_100')

# When downloading 3D connectivity data volumes, what resolution do you want (in microns)?  
# Options are: 10, 25, 50, 100
resolution_um=10

# Downsampling factor
stride = 4

# The manifest file is a simple JSON file that keeps track of all of
# the data that has already been downloaded onto the hard drives.
# If you supply a relative path, it is assumed to be relative to your
# current working directory.
manifest_file = os.path.join(drive_path, "manifest.json")

# Start processing data
mcc = MouseConnectivityCache(manifest_file = manifest_file,
                             resolution = resolution_um)
ontology = mcc.get_ontology()
# Injection structure of interest
isocortex = ontology['Isocortex']

# open up a pandas dataframe of all of the experiments
experiments = mcc.get_experiments(dataframe = True, 
                                  injection_structure_ids = [isocortex['id'].values[0]], 
                                  cre = False)
print "%d total experiments" % len(experiments)

def downsample(arr, stride):
    assert type(stride) is int, "stride should be integer"
    return arr[0::stride, 0::stride]

/home/kamdh/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:26: VisibleDeprecationWarning: Function get_ontology is deprecated. Use get_structure_tree instead.


126 total experiments


In [7]:
## Laplacians
view_paths_fn = os.path.join(os.getenv('HOME'), 'work/allen/data/TopView/top_view_paths_10.h5')
view_paths_file = h5py.File(view_paths_fn, 'r')
view_lut = view_paths_file['view lookup'][:]
view_paths = view_paths_file['paths'][:]
view_paths_file.close()

## Compute size of each path to convert path averages to sums
norm_lut = np.zeros(view_lut.shape, dtype=int)
ind = np.where(view_lut != -1)
ind = zip(ind[0], ind[1])
for curr_ind in ind:
    curr_path_id = view_lut[curr_ind]
    curr_path = view_paths[curr_path_id, :]
    norm_lut[curr_ind] = np.sum(curr_path > 0)

In [8]:
view_lut = downsample(view_lut, stride)
#norm_lut = downsample(norm_lut, stride)

In [9]:
print norm_lut.shape

(1320, 1140)


In [10]:
data_mask = np.where(view_lut != -1)
# Right indices
right = np.zeros(view_lut.shape, dtype=bool)
right[:, int(view_lut.shape[1]/2):] = True
# Right hemisphere data
hemi_R_mask = np.where(np.logical_and(view_lut != -1, right))
# Left hemisphere data
hemi_L_mask = np.where(np.logical_and(view_lut != -1, np.logical_not(right)))

nx = len(hemi_R_mask[0])
ny = len(data_mask[0])

In [11]:
print nx, ny

22377 44478


In [12]:
def laplacian_2d(mask):
    '''
    Generate the laplacian matrix for a given region's voxels. This is the 
    graph laplacian of the neighborhood graph.

    Parameters
    ----------
      mask

    Returns
    -------
      L: num voxel x num voxel laplacian csc_matrix in same order as voxels
        in the region mask
    '''
    def possible_neighbors(vox):
        neighbors = np.tile(vox, (4,1))
        neighbors += [[1,0],
                      [0,1],
                      [-1,0],
                      [0,-1]]
        return neighbors

    import scipy.sparse as sp
    voxels = np.array(mask).T
    num_vox = len(mask[0])
    # num_vox=voxels.shape[0]
    vox_lookup = {tuple(vox): idx for idx, vox in enumerate(voxels)}
    L = sp.lil_matrix((num_vox, num_vox))
    for idx, vox in enumerate(voxels):
        candidates = possible_neighbors(vox)
        deg = 0
        for nei in candidates:
            try:
                idx_nei = vox_lookup[ tuple(nei) ]
                deg += 1
                L[idx, idx_nei] = 1
            except KeyError:
                continue
        L[idx, idx] = -deg
    return L.tocsc()

In [13]:
Lx = laplacian_2d(hemi_R_mask)
Ly = laplacian_2d(data_mask)
print Ly.shape
print Ly.nnz
print ny

(44478, 44478)
221328
44478


In [ ]:
X = np.zeros((nx, len(experiments)))
Y = np.zeros((ny, len(experiments)))
Omega = np.zeros((ny, len(experiments)))
expt_drop_list = []
t0 = time.time()
#eid = experiments.iloc[5].id
#row = experiments.iloc[5]
index = 0
for eid, row in experiments.iterrows():
    print "\nRow %d\nProcessing experiment %d" % (index,eid)
    print row
    data_dir = os.path.join(os.getenv('HOME'),
                            "work/allen/data/sdk_new_100/experiment_%d/" % eid)
    # get and remap injection data
    in_fn = data_dir + "injection_density_top_view_%d.nrrd" % int(resolution_um)
    print "reading " + in_fn
    in_d_s_full = nrrd.read(in_fn)[0]
    flat_vol = np.nansum(in_d_s_full * norm_lut) * (10./1000.)**3
    expt_union = mcc.get_experiment_structure_unionizes(eid, hemisphere_ids = [3], is_injection = True,
                                                        structure_ids = [isocortex['id'].values[0]])
    full_vol = float(expt_union['projection_volume'])
    in_d_s = downsample(in_d_s_full, stride)
    # get and remap projection data
    pr_fn = data_dir + "projection_density_top_view_%d.nrrd" % int(resolution_um)
    print "reading " + pr_fn
    pr_d_s = downsample(nrrd.read(pr_fn)[0], stride)
    # fill matrices
    X[:, index] = in_d_s[hemi_R_mask]
    Y[:, index] = pr_d_s[data_mask]
    this_Omega = (in_d_s[data_mask] > Omega_thresh).astype(int)
    Omega[:, index] = this_Omega
    # drop experiments without much injection volume
    if np.abs(flat_vol - full_vol) / full_vol * 100 > volume_fraction:
        print "warning, dropping experiment"
        print "flat_vol = %f\nfull_vol = %f" % (flat_vol, full_vol)
        expt_drop_list.append(index)
    index += 1
t1 = time.time()
total = t1 - t0
print "%0.1f minutes elapsed" % (total/60.)


Row 0
Processing experiment 180435652
gender                                                                   M
id                                                               180435652
injection-coordinates                                   [7820, 4250, 9870]
injection-structures     [{u'abbreviation': u'TEa', u'color': u'15B0B3'...
product-id                                                               5
strain                                                            C57BL/6J
structure-abbrev                                                       ECT
structure-color                                                     0D9F91
structure-id                                                           895
structure-name                                             Ectorhinal area
transgenic-line                                                           
Name: 180435652, dtype: object
reading /home/kamdh/work/allen/data/sdk_new_100/experiment_180435652/injection_density_top_view_10.nrrd
r

reading /home/kamdh/work/allen/data/sdk_new_100/experiment_180720175/projection_density_top_view_10.nrrd

Row 8
Processing experiment 180717881
gender                                                                   M
id                                                               180717881
injection-coordinates                                   [4580, 3610, 8670]
injection-structures     [{u'abbreviation': u'SSp-m', u'color': u'18806...
product-id                                                               5
strain                                                            C57BL/6J
structure-abbrev                                                     SSp-m
structure-color                                                     188064
structure-id                                                           345
structure-name                           Primary somatosensory area, mouth
transgenic-line                                                           
Name: 180717881, dtype: object


warning, dropping experiment
flat_vol = 0.185936
full_vol = 0.388083

Row 16
Processing experiment 141603190
gender                                                                   M
id                                                               141603190
injection-coordinates                                   [4100, 1810, 6110]
injection-structures     [{u'abbreviation': u'ACAd', u'color': u'40A666...
product-id                                                               5
strain                                                            C57BL/6J
structure-abbrev                                                       MOs
structure-color                                                     1F9D5A
structure-id                                                           993
structure-name                                        Secondary motor area
transgenic-line                                                           
Name: 141603190, dtype: object
reading /home/kamdh/work/allen/data

reading /home/kamdh/work/allen/data/sdk_new_100/experiment_170721670/projection_density_top_view_10.nrrd
warning, dropping experiment
flat_vol = 0.000001
full_vol = 0.204924

Row 25
Processing experiment 157710335
gender                                                                   M
id                                                               157710335
injection-coordinates                                   [2440, 2750, 7050]
injection-structures     [{u'abbreviation': u'FRP', u'color': u'268F45'...
product-id                                                               5
strain                                                            C57BL/6J
structure-abbrev                                                       MOs
structure-color                                                     1F9D5A
structure-id                                                           993
structure-name                                        Secondary motor area
transgenic-line                     

reading /home/kamdh/work/allen/data/sdk_new_100/experiment_100142655/projection_density_top_view_10.nrrd
warning, dropping experiment
flat_vol = 0.097064
full_vol = 0.175529

Row 34
Processing experiment 112514202
gender                                                                   M
id                                                               112514202
injection-coordinates                                   [4250, 2450, 5860]
injection-structures     [{u'abbreviation': u'ACAd', u'color': u'40A666...
product-id                                                               5
strain                                                            C57BL/6J
structure-abbrev                                                      ACAv
structure-color                                                     40A666
structure-id                                                            48
structure-name                       Anterior cingulate area, ventral part
transgenic-line                     

reading /home/kamdh/work/allen/data/sdk_new_100/experiment_158314278/projection_density_top_view_10.nrrd
warning, dropping experiment
flat_vol = 0.102000
full_vol = 0.273547

Row 43
Processing experiment 112791318
gender                                                                   M
id                                                               112791318
injection-coordinates                                   [6370, 1630, 7470]
injection-structures     [{u'abbreviation': u'SSp-ll', u'color': u'1880...
product-id                                                               5
strain                                                            C57BL/6J
structure-abbrev                                                    SSp-ll
structure-color                                                     188064
structure-id                                                           337
structure-name                      Primary somatosensory area, lower limb
transgenic-line                     

reading /home/kamdh/work/allen/data/sdk_new_100/experiment_307137980/projection_density_top_view_10.nrrd
warning, dropping experiment
flat_vol = 0.226466
full_vol = 0.475240

Row 52
Processing experiment 309113907
gender                                                                   M
id                                                               309113907
injection-coordinates                                   [8690, 1290, 7780]
injection-structures     [{u'abbreviation': u'VISp', u'color': u'08858C...
product-id                                                               5
strain                                                            C57BL/6J
structure-abbrev                                                      VISp
structure-color                                                     08858C
structure-id                                                           385
structure-name                                         Primary visual area
transgenic-line                     

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

fig = plt.figure(figsize = (20,20))
ax = fig.add_subplot(121)
h = ax.imshow(in_d_s)
fig.colorbar(h)

#fig2 = plt.figure(figsize = (10,10))
ax2 = fig.add_subplot(122)
h2 = ax2.imshow(pr_d_s)
ax2.plot([np.floor(285/2.), np.floor(285/2.)], [0, 330], 'k-', linewidth=0.2 )
#fig2.colorbar(h2)

In [ ]:
vol1 = np.nansum(nrrd.read(in_fn)[0] * norm_lut)
print vol1
vol2 = np.sum(mcc.get_injection_density(eid)[0])
print vol2
print np.abs(vol1 - vol2)/vol2

In [ ]:
tmp = mcc.get_structure_unionizes()

In [ ]:
tmp = downsample(pr_d_s, stride)[hemi_R_mask]
plt.hist( tmp[tmp!=0])

In [ ]:
output_dir = os.path.join(os.getenv('HOME'), 'work/allen/data/2d_test')
from scipy.io import mmwrite
mmwrite(os.path.join(output_dir, "Lx.mtx"), Lx)
mmwrite(os.path.join(output_dir, "Ly.mtx"), Ly)

In [ ]:
    expt_union = mcc.get_experiment_structure_unionizes(eid,
                                                        hemisphere_ids = [3],
                                                        is_injection = True,
                                                        structure_ids = [ontology['grey']['id'].values[0]])

In [ ]:
expt_union

In [ ]:
expt_union['projection_volume']

In [ ]:
flat_vol = vol1 * (10/1000.)**3
full_vol = float(expt_union['projection_volume'])

In [ ]:
np.abs(flat_vol - full_vol)/full_vol*100

In [15]:
volume_fraction = 30
Omega_thresh = 0.5

In [ ]:
print len(expt_drop_list)
print len(experiments)

In [19]:
print "drop list: " + str(expt_drop_list)


drop list: [0, 1, 2, 3, 4, 6, 8, 9, 10, 11, 12, 14, 15, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 109, 110, 111, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125]


In [22]:
np.array(data_mask).T

array([[ 46, 109],
       [ 46, 110],
       [ 46, 111],
       ..., 
       [259, 225],
       [259, 226],
       [259, 227]])

In [24]:
hemi_R_mask

(array([ 46,  46,  46, ..., 259, 259, 259]),
 array([150, 151, 152, ..., 225, 226, 227]))

In [25]:
view_lut

array([[-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1],
       ..., 
       [-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1]], dtype=int32)